In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. Load Data
train_df = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

# 2. Feature Engineering (paste your entire feature engineering block here)
train_df['Sex_Reversed'] = train_df['Sex'].map({'male': 1, 'female': 0})
test_df['Sex_Reversed'] = test_df['Sex'].map({'male': 1, 'female': 0})

train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

train_df['Heart_Rate_pct'] = train_df['Heart_Rate'] / (220 - train_df['Age'])
test_df['Heart_Rate_pct'] = test_df['Heart_Rate'] / (220 - test_df['Age'])

train_df['BMI'] = train_df['Weight'] / (train_df['Height']/100)**2
test_df['BMI'] = test_df['Weight'] / (test_df['Height']/100)**2

train_df['BMR'] = np.where(
    train_df['Sex'] == 'female',
    10 * train_df['Weight'] + 6.25 * train_df['Height'] - 5 * train_df['Age'] - 161,
    10 * train_df['Weight'] + 6.25 * train_df['Height'] - 5 * train_df['Age'] + 5
)
test_df['BMR'] = np.where(
    test_df['Sex'] == 'female',
    10 * test_df['Weight'] + 6.25 * test_df['Height'] - 5 * test_df['Age'] - 161,
    10 * test_df['Weight'] + 6.25 * test_df['Height'] - 5 * test_df['Age'] + 5
)

train_df['TSI'] = 5 * ((train_df['Body_Temp'] - 36.5) / (41.5 - 36.5)) + 5 * ((train_df['Heart_Rate'] - 60) / ((220 - train_df['Age']) - 60))
train_df['RPE'] = train_df['Heart_Rate_pct'] + 0.1 * (train_df['Body_Temp'] - 37)
train_df['FI'] = (train_df['Heart_Rate_pct'] ** 2) / train_df['Duration']
train_df['CLI'] = (train_df['Heart_Rate'] * train_df['Duration']) / train_df['Weight']
train_df['TLI'] = ((train_df['Body_Temp'] - 36.6) ** 2) * train_df['Duration']
train_df['AMI'] = (train_df['BMR'] * train_df['Heart_Rate_pct']) / train_df['Duration']
train_df['AWI'] = (train_df['Duration'] * train_df['Heart_Rate_pct']) / train_df['Age']
train_df['WLI'] = train_df['Heart_Rate'] * train_df['Duration'] * train_df['Weight']
train_df['VO2_Proxy'] = np.where(
    train_df['Sex'] == 'female',
    (0.85 * train_df['Duration']) / (train_df['Heart_Rate_pct'] * train_df['Age']),
    (1.00 * train_df['Duration']) / (train_df['Heart_Rate_pct'] * train_df['Age']),
)
test_df['TSI'] = 5 * ((test_df['Body_Temp'] - 36.5) / (41.5 - 36.5)) + 5 * ((test_df['Heart_Rate'] - 60) / ((220 - test_df['Age']) - 60))
test_df['RPE'] = test_df['Heart_Rate_pct'] + 0.1 * (test_df['Body_Temp'] - 37)
test_df['FI'] = (test_df['Heart_Rate_pct'] ** 2) / test_df['Duration']
test_df['CLI'] = (test_df['Heart_Rate'] * test_df['Duration']) / test_df['Weight']
test_df['TLI'] = ((test_df['Body_Temp'] - 36.6) ** 2) * test_df['Duration']
test_df['AMI'] = (test_df['BMR'] * test_df['Heart_Rate_pct']) / test_df['Duration']
test_df['AWI'] = (test_df['Duration'] * test_df['Heart_Rate_pct']) / test_df['Age']
test_df['WLI'] = test_df['Heart_Rate'] * test_df['Duration'] * test_df['Weight']
test_df['VO2_Proxy'] = np.where(
    test_df['Sex'] == 'female',
    (0.85 * test_df['Duration']) / (test_df['Heart_Rate_pct'] * test_df['Age']),
    (1.00 * test_df['Duration']) / (test_df['Heart_Rate_pct'] * test_df['Age']),
)

train_df['Duration_HR'] = train_df['Duration'] * train_df['Heart_Rate']
test_df['Duration_HR'] = test_df['Duration'] * test_df['Heart_Rate']

train_df['Duration2_HR'] = (train_df['Duration'])**2 * train_df['Heart_Rate']
test_df['Duration2_HR'] = (test_df['Duration'])**2 * test_df['Heart_Rate']

train_df['Intensity'] = train_df['Heart_Rate'] / train_df['Duration']
test_df['Intensity'] = test_df['Heart_Rate'] / test_df['Duration']

for f1 in ['Duration', 'Heart_Rate', 'Body_Temp']:
        for f2 in ['Sex', 'Sex_Reversed']:
            train_df[f'{f1}_x_{f2}'] = train_df[f1] * train_df[f2]
for f1 in ['Duration', 'Heart_Rate', 'Body_Temp']:
        for f2 in ['Sex', 'Sex_Reversed']:
            test_df[f'{f1}_x_{f2}'] = test_df[f1] * test_df[f2]

train_df['Body_Temp'] = train_df['Body_Temp'] - 37.0
test_df['Body_Temp'] = test_df['Body_Temp'] - 37.0

# for col in ['Height', 'Weight', 'Heart_Rate', 'Body_Temp']:
#         for agg in ['min', 'max']:
#             agg_val = train_df.groupby('Sex')[col].agg(agg).rename(f'Sex_{col}_{agg}')
#             train_df = train_df.merge(agg_val, on='Sex', how='left')
# for col in ['Height', 'Weight', 'Heart_Rate', 'Body_Temp']:
#         for agg in ['min', 'max']:
#             agg_val = test_df.groupby('Sex')[col].agg(agg).rename(f'Sex_{col}_{agg}')
#             test_df = test_df.merge(agg_val, on='Sex', how='left')

# Calculate 'Heart_Rate_Ratio' for the training data
train_df['Heart_Rate_Ratio'] = train_df['Heart_Rate'] / train_df['Age']
# Calculate 'Heart_Rate_Ratio' for the testing data
test_df['Heart_Rate_Ratio'] = test_df['Heart_Rate'] / test_df['Age']

# Calculate 'Weight_x_Duration' for the training data
train_df['Weight_x_Duration'] = train_df['Weight'] * train_df['Duration']
# Calculate 'Weight_x_Duration' for the testing data
test_df['Weight_x_Duration'] = test_df['Weight'] * test_df['Duration']

# Calculate 'Height_x_Duration' for the training data
train_df['Height_x_Duration'] = train_df['Height'] * train_df['Duration']
# Calculate 'Height_x_Duration' for the testing data
test_df['Height_x_Duration'] = test_df['Height'] * test_df['Duration']

# Calculate 'Weight_x_Height' for the training data
train_df['Weight_x_Height'] = train_df['Weight'] * train_df['Height']
# Calculate 'Weight_x_Height' for the testing data
test_df['Weight_x_Height'] = test_df['Weight'] * test_df['Height']

# Calculate 'Weight_x_Intensity' for the training data
train_df['Weight_x_Intensity'] = train_df['Weight'] * train_df['Intensity']
# Calculate 'Weight_x_Intensity' for the testing data
test_df['Weight_x_Intensity'] = test_df['Weight'] * test_df['Intensity']

# Calculate 'Height_x_Intensity' for the training data
train_df['Height_x_Intensity'] = train_df['Height'] * train_df['Intensity']
# Calculate 'Height_x_Intensity' for the testing data
test_df['Height_x_Intensity'] = test_df['Height'] * test_df['Intensity']

train_df.drop(columns=['Sex_Reversed'], inplace=True)
test_df.drop(columns=['Sex_Reversed'], inplace=True)

train_df.drop(columns=['id'], inplace=True)
test_df.drop(columns=['id'], inplace=True)


# After feature engineering:
X = train_df.drop(columns=["Calories"])
y = train_df["Calories"]
X_test = test_df.copy()

# 3. Train XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.07,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X, y)
xgb_train_pred = xgb_model.predict(X)
xgb_test_pred = xgb_model.predict(X_test)

# 4. Add XGB prediction as input feature
X["xgb_pred"] = xgb_train_pred
X_test["xgb_pred"] = xgb_test_pred

# 5. Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
y = y.values.reshape(-1, 1)

# 6. Define Residual Block NN
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )
        self.activation = nn.ReLU()

    def forward(self, x):
        return self.activation(x + self.block(x))

class MetaNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.middle = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.out = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.input_layer(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.middle(x)
        return self.out(x)

# 7. Cross-validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X_scaled.shape[1]
fold_options = [5, 10, 15, 30]

for n_splits in fold_options:
    print(f"\n🧪 Training with {n_splits}-Fold CV...")
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    test_preds = np.zeros((X_test.shape[0],))

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
        print(f" Fold {fold + 1}/{n_splits}")
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = MetaNN(input_dim).to(device)
        optimizer = optim.AdamW(model.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
        loss_fn = nn.MSELoss()

        train_loader = DataLoader(
            TensorDataset(
                torch.tensor(X_train, dtype=torch.float32),
                torch.tensor(y_train, dtype=torch.float32)
            ),
            batch_size=64,
            shuffle=True
        )

        best_loss = np.inf
        best_model = None

        for epoch in range(60):
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                pred = model(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                optimizer.step()
            scheduler.step()

            # Validation
            model.eval()
            with torch.no_grad():
                val_preds = model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()
            val_rmse = mean_squared_error(y_val, val_preds, squared=False)
            if val_rmse < best_loss:
                best_loss = val_rmse
                best_model = model.state_dict()

        # Load best model and predict
        model.load_state_dict(best_model)
        model.eval()
        test_fold_pred = model(torch.tensor(X_test_scaled, dtype=torch.float32).to(device)).cpu().detach().numpy().ravel()
        test_preds += test_fold_pred / n_splits

    # Save submission
    submission = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")
    test_df_id = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")["id"]
    submission["id"] = test_df_id
    submission["predictions"] = test_preds
    submission.to_csv(f"submission_nn_stack_k{n_splits}.csv", index=False)
    print(f"📁 Saved: submission_nn_stack_k{n_splits}.csv")
